In [4]:
import pandas as pd
import my_file
import sqlalchemy
import requests
import datetime

In [2]:
schema="gans"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password=my_file.mysql_password # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [3]:
airports_df = pd.read_sql_table('airports',con=con)
airports_df

,ICAO,city_id
0,EDDB,1
1,EGLL,2


In [8]:
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
tomorrow = tomorrow.strftime('%Y-%m-%d')
tomorrow

'2023-09-07'

In [10]:
time_from = "00:00"
time_to = "11:59"

In [12]:
flights_dict = {'Arrival_airport':[],
                'Departure_airport_icao':[],
                'Departure_airport_city':[],
                'Departure_terminal':[],
                'Arrival_time_local':[]}

for index, row in airports_df.iterrows():

    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['ICAO']}/{tomorrow}T{time_from}/{tomorrow}T{time_to}"

    querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

    headers = {
	"X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    flight_json = response.json()
    for flight in flight_json['arrivals']:
        flights_dict['Arrival_airport'].append(row['ICAO'])
        flights_dict['Departure_airport_icao'].append(flight['departure']['airport']['icao'])
        flights_dict['Departure_airport_city'].append(flight['departure']['airport']['name'])
        flights_dict['Arrival_time_local'].append(flight['arrival']['scheduledTimeLocal'])
        try:
            flights_dict['Departure_terminal'].append(flight['departure']['terminal'])
        except:
            flights_dict['Departure_terminal'].append('unknown')

flights_df = pd.DataFrame(flights_dict)
# print(response.json())

In [15]:
flights_df['Arrival_time_local'] = flights_df['Arrival_time_local'].str[:-6]

In [16]:
flights_df

,Arrival_airport,Departure_airport_icao,Departure_airport_city,Departure_terminal,Arrival_time_local
0,EDDB,LTBJ,İzmir,I,2023-09-07 05:45
1,EDDB,HEMA,Marsa Alam,unknown,2023-09-07 06:20
2,EDDB,LBWN,Varna,unknown,2023-09-07 06:40
3,EDDB,KJFK,New York,4,2023-09-07 07:05
4,EDDB,LFPG,Paris,unknown,2023-09-07 07:55
...,...,...,...,...,...
1188,EGLL,CYYC,Calgary,unknown,2023-09-07 11:50
1189,EGLL,KJFK,New York,4,2023-09-07 11:10
1190,EGLL,KSFO,San Francisco,ITM,2023-09-07 11:10
1191,EGLL,KATL,Atlanta,unknown,2023-09-07 11:10


In [17]:
flights_df.to_sql('flights',con=con,if_exists='append',index=False)

1193